# Otimização de Modelo Pré-Treinado para Detecção de Fraudes em Cartões de Crédito

Giovanna Furlan Torres

Turma 4 - Sistemas de Informação

## Problemática

Otimizar um modelo de rede neural pré-treinado para detecção de fraudes em cartões de crédito. Aplicar técnicas avançadas de ajuste fino de hiperparâmetros, como grid search e random search, com o objetivo de aprimorar as métricas de desempenho do modelo, incluindo precisão, recall, F1-score e AUC-ROC. A atividade também exige uma comparação entre o modelo otimizado e o modelo original, permitindo avaliar o impacto das modificações nos hiperparâmetros sobre o desempenho geral.

# Setup

A configuração de setup é o processo de preparar e organizar o ambiente para uso.

## Bibliotecas

Essa etapa envolve a instalação de bibliotecas e configuração de outros ajustes necessários.

In [1]:
import numpy as np
import pandas as pd
import gdown
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

In [3]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.4 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from keras_tuner import RandomSearch

## Importação dos dados

Para garantir o acesso aos dados por todos, sem a necessidade de permissões, foi utilizado o gdown para consumir esse dataset.

In [5]:
arquivo_destino_colab = "dataset.csv"

doc_id = "1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl"

URL = f"https://drive.google.com/uc?id={doc_id}"

gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl
From (redirected): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl&confirm=t&uuid=89ef5b80-7f7e-4caa-80f7-f89fff27dca5
To: /content/dataset.csv
100%|██████████| 151M/151M [00:04<00:00, 36.4MB/s]


'dataset.csv'

In [6]:
dados = pd.read_csv(arquivo_destino_colab)
dados

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


## Informações Gerais

In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

### **Sumário Estatístico**

Abaixo, apresenta-se uma visão geral dos dados olímpicos:

In [8]:
num_amostras, num_features = dados.shape

- **Número de amostras:** Indica a quantidade total de registros no conjunto de dados.

In [9]:
print(f"Número de amostras: {num_amostras}")

Número de amostras: 284807


- **Número de features:** Mostra o total de variáveis presentes em cada amostra.


In [10]:
print(f"Número de features: {num_features}")

Número de features: 31


- **Tipos de dados das features:** Exibe os tipos de dados associados a cada coluna do DataFrame.

In [11]:
print("\nTipos de dados das features:")
print(dados.dtypes)


Tipos de dados das features:
Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object


- **Descrição estatística das features numéricas:** Fornece estatísticas descritivas como média, desvio padrão, valores mínimo e máximo, entre outros, para as variáveis numéricas.

In [12]:
print("\nDescrição estatística das features numéricas:")
print(dados.describe())


Descrição estatística das features numéricas:
                Time            V1            V2            V3            V4  \
count  284807.000000  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean    94813.859575  1.168375e-15  3.416908e-16 -1.379537e-15  2.074095e-15   
std     47488.145955  1.958696e+00  1.651309e+00  1.516255e+00  1.415869e+00   
min         0.000000 -5.640751e+01 -7.271573e+01 -4.832559e+01 -5.683171e+00   
25%     54201.500000 -9.203734e-01 -5.985499e-01 -8.903648e-01 -8.486401e-01   
50%     84692.000000  1.810880e-02  6.548556e-02  1.798463e-01 -1.984653e-02   
75%    139320.500000  1.315642e+00  8.037239e-01  1.027196e+00  7.433413e-01   
max    172792.000000  2.454930e+00  2.205773e+01  9.382558e+00  1.687534e+01   

                 V5            V6            V7            V8            V9  \
count  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean   9.604066e-16  1.487313e-15 -5.556467e-16  1.213481e-16 -2.406331e-1

- **Valores ausentes por coluna:** Lista a quantidade de valores ausentes em cada coluna do DataFrame.

In [13]:
print("\nValores ausentes por coluna:")
print(dados.isnull().sum())


Valores ausentes por coluna:
Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


- **Valores únicos em cada coluna:** Apresenta o número de valores distintos encontrados em cada coluna.

In [14]:
print("\nValores únicos em cada coluna:")
print(dados.nunique())


Valores únicos em cada coluna:
Time      124592
V1        275663
V2        275663
V3        275663
V4        275663
V5        275663
V6        275663
V7        275663
V8        275663
V9        275663
V10       275663
V11       275663
V12       275663
V13       275663
V14       275663
V15       275663
V16       275663
V17       275663
V18       275663
V19       275663
V20       275663
V21       275663
V22       275663
V23       275663
V24       275663
V25       275663
V26       275663
V27       275663
V28       275663
Amount     32767
Class          2
dtype: int64


## Normalização

A normalização das colunas é necessária para garantir que os valores estejam em uma escala comum, facilitando o desempenho do modelo que são sensíveis a diferentes escalas de dados. Ao normalizar, transforma-se os valores para um intervalo entre 0 e 1.

In [15]:
# Normalização da coluna Amount
scaler = MinMaxScaler()

# Aplicando o scaler somente à coluna Amount
dados['Amount'] = scaler.fit_transform(dados[['Amount']])
dados['Time'] = scaler.fit_transform(dados[['Time']])
dados

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000000,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.005824,0
1,0.000000,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0.000105,0
2,0.000006,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0.014739,0
3,0.000006,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.004807,0
4,0.000012,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.002724,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,0.999965,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.000030,0
284803,0.999971,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,0.000965,0
284804,0.999977,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,0.002642,0
284805,0.999977,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,0.000389,0


# Atenção

A análise dos dados revelou que, além do desbalanceamento das classes, os modelos estão apresentando uma acurácia excessivamente alta, indicando possíveis problemas. Além disso, a busca pelos melhores hiperparâmetros está levando ao overfitting, prejudicando a performance do modelo. Os códigos a seguir detalham melhor esses aspectos.

# Modelo sem alteração no dataset

O primeiro modelo a ser testado não terá nenhuma modificação nos dados, aplicando apenas a função de ativação ReLU nas camadas ocultas e sigmoid na camada de saída. Esse modelo serve para demonstrar o desempenho com os dados em seu estado original.

In [16]:
# Separando as features e o target
X = dados.drop(columns=['Class'])
y = dados['Class']

# Dividindo o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criando o modelo
model = Sequential([
    Dense(16, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Avaliando o modelo nos dados de teste
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculando as métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
4985/4985 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9869 - loss: 0.0544 - val_accuracy: 0.9994 - val_loss: 0.0032
Epoch 2/10
4985/4985 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9994 - loss: 0.0030 - val_accuracy: 0.9994 - val_loss: 0.0030
Epoch 3/10
4985/4985 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9994 - loss: 0.0034 - val_accuracy: 0.9994 - val_loss: 0.0030
Epoch 4/10
4985/4985 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9995 - loss: 0.0026 - val_accuracy: 0.9994 - val_loss: 0.0028
Epoch 5/10
4985/4985 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9994 - loss: 0.0030 - val_accuracy: 0.9994 - val_loss: 0.0027
Epoch 6/10
4985/4985 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9994 - loss: 0.0027 - val_accuracy: 0.9994 - val_loss: 0.0028
Epoch 7/10
4985/4985 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9994 - loss: 0.0027 - val_accuracy: 0.9993 - val_loss: 0.0027
Epoch 8/10
4985/4985 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9995 - loss: 0

In [17]:
# Exibindo os resultados
print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'AUC-ROC: {roc_auc:.4f}')

Acurácia: 0.9994
Precisão: 0.8188
Recall: 0.8309
F1-Score: 0.8248
AUC-ROC: 0.9899


In [18]:
# Calculando a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Definindo os rótulos
labels = ['Classe 0', 'Classe 1']

# Criando a figura
fig = ff.create_annotated_heatmap(z=cm, x=labels, y=labels, colorscale='Blues', showscale=True)

fig.update_layout(
    title='Matriz de Confusão - Modelo 1',
    xaxis=dict(title='Valores Preditos'),
    yaxis=dict(title='Valores Reais')
)

# Exibindo a matriz
fig.show()

Os resultados do primeiro modelo indicam:

- **Acurácia (0.9994)**: O modelo acertou 99,94% das previsões, o que sugere um bom desempenho em termos gerais. No entanto, como a acurácia pode ser afetada por um desbalanceamento entre as classes.

- **Precisão (0.8028)**: Entre todas as transações que o modelo classificou como fraudulentas, 80,28% realmente eram fraudes. Isso é importante, especialmente em cenários onde falsos positivos podem gerar custos ou inconveniências.

- **Recall (0.8382)**: O modelo identificou corretamente 83,82% das fraudes reais. Um bom recall garante que o modelo capture a maioria das fraudes, minimizando o risco de deixar passarem despercebidas.

- **F1-Score (0.8201)**: Combinando precisão e recall, o F1-Score de 82,01% sugere que o modelo consegue capturar fraudes, enquanto ainda mantém um número gerenciável de falsos positivos.

- **AUC-ROC (0.9924)**: A área sob a curva ROC de 99,24% indica que o modelo tem uma excelente capacidade de diferenciar entre transações fraudulentas e legítimas em vários limiares de decisão.


# Modelo com Balanceamento de Classes

Como observado na última matriz de confusão, as classes estão desbalanceadas, conforme demonstrado no código a seguir. Portanto, o objetivo deste modelo é executar a versão com as classes já balanceadas e analisar seus resultados.

In [19]:
dados['Class'].value_counts()

,count
Class,
0,284315
1,492


Para balancear os dados, foram utilizadas as seguintes abordagens:

- **SMOTE (Synthetic Minority Over-sampling Technique)**: Técnica utilizada para gerar exemplos sintéticos da classe minoritária, ajudando a equilibrar o dataset e melhorar o desempenho do modelo.

- **Padronização das Features**: As features foram padronizadas usando o `StandardScaler`, garantindo que todas estejam na mesma escala.

In [20]:
# Aplicando o SMOTE para balancear as classes no conjunto de treinamento
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Padronizando as features
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Criando o modelo
model = Sequential([
    Dense(16, input_shape=(X_train_balanced.shape[1],), activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo
model.fit(X_train_balanced, y_train_balanced, epochs=10, batch_size=32, validation_split=0.2)

# Avaliando o modelo nos dados de teste
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculando as métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



9951/9951 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9576 - loss: 0.1192 - val_accuracy: 0.9862 - val_loss: 0.0372
Epoch 2/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9870 - loss: 0.0371 - val_accuracy: 0.9816 - val_loss: 0.0470
Epoch 3/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9932 - loss: 0.0219 - val_accuracy: 0.9965 - val_loss: 0.0181
Epoch 4/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9952 - loss: 0.0171 - val_accuracy: 0.9989 - val_loss: 0.0116
Epoch 5/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9960 - loss: 0.0143 - val_accuracy: 0.9976 - val_loss: 0.0117
Epoch 6/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9965 - loss: 0.0124 - val_accuracy: 0.9994 - val_loss: 0.0067
Epoch 7/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9969 - loss: 0.0112 - val_accuracy: 0.9986 - val_loss: 0.0072
Epoch 8/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9973 - loss: 0.0101 - val

In [21]:
# Exibindo os resultados
print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'AUC-ROC: {roc_auc:.4f}')

Acurácia: 0.9971
Precisão: 0.3362
Recall: 0.8529
F1-Score: 0.4823
AUC-ROC: 0.9775


Após o balanceamento com SMOTE, as métricas do modelo foram ajustadas, refletindo as mudanças esperadas em situações de classes desbalanceadas:

- **Acurácia (0.9974)**: A acurácia ainda é alta, mas apresentou uma leve redução em relação ao modelo anterior. Isso sugere que, apesar do bom desempenho geral, o modelo agora é mais sensível à classe minoritária, refletindo o impacto do balanceamento.

- **Precisão (0.3620)**: A precisão caiu, indicando que o modelo está classificando mais transações como fraudulentas, resultando em um aumento nos falsos positivos. Esse é um efeito esperado ao priorizar a detecção da classe minoritária, em detrimento da precisão.

- **Recall (0.8676)**: O recall aumentou, mostrando que o modelo agora captura uma maior proporção de fraudes reais. Isso é positivo, pois reduz a quantidade de fraudes não detectadas pelo modelo.

- **F1-Score (0.5108)**: O F1-Score, que equilibra precisão e recall, apresenta um desempenho moderado. O aumento do recall é evidente, mas a precisão mais baixa resulta em um F1-Score que busca evitar falsos positivos.

- **AUC-ROC (0.9771)**: A AUC-ROC permanece alta, indicando que o modelo continua a ter uma boa capacidade de distinguir entre fraudes e não fraudes.

Isso sugere que o balanceamento ajudou a melhorar a sensibilidade do modelo sem comprometer sua capacidade de separação.

# Modelo com Hiperparâmetros

Neste código, o Keras Tuner utiliza a técnica de Random Search para encontrar os melhores hiperparâmetros para o modelo. O build_model define a arquitetura do modelo com hiperparâmetros ajustáveis, enquanto o RandomSearch configura o processo de busca pelos melhores parâmetros. O SMOTE é aplicado para balancear as classes no conjunto de treinamento, e as features são padronizadas antes do treinamento do modelo.

In [22]:
# Função para criar o modelo
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units1', min_value=8, max_value=64, step=8),
                    activation='relu',
                    input_shape=(X_train_balanced.shape[1],)))
    model.add(Dense(units=hp.Int('units2', min_value=8, max_value=64, step=8),
                    activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Configurando o Keras Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Aplicando o SMOTE para balancear as classes no conjunto de treinamento
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Padronizando as features (escalonamento)
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Treinando o modelo
tuner.search(X_train_balanced, y_train_balanced, epochs=10, validation_split=0.2)

# Obtendo os melhores hiperparâmetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f'Best Hyperparameters: {best_hps.values}')

Trial 10 Complete [00h 04m 30s]
val_accuracy: 0.9992839694023132

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 46m 04s
Best Hyperparameters: {'units1': 56, 'units2': 24, 'optimizer': 'adam'}


In [23]:
# Construindo o modelo com os melhores hiperparâmetros
model = tuner.hypermodel.build(best_hps)
model.fit(X_train_balanced, y_train_balanced, epochs=10, validation_split=0.2)

# Avaliando o modelo nos dados de teste
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)


# Calculando as métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

Epoch 1/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9702 - loss: 0.0830 - val_accuracy: 0.9996 - val_loss: 0.0056
Epoch 2/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9970 - loss: 0.0111 - val_accuracy: 0.9998 - val_loss: 0.0023
Epoch 3/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9980 - loss: 0.0072 - val_accuracy: 0.9991 - val_loss: 0.0026
Epoch 4/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9985 - loss: 0.0054 - val_accuracy: 0.9969 - val_loss: 0.0113
Epoch 5/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9987 - loss: 0.0045 - val_accuracy: 1.0000 - val_loss: 2.1939e-04
Epoch 6/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.9990 - loss: 0.0035 - val_accuracy: 1.0000 - val_loss: 2.6985e-04
Epoch 7/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9992 - loss: 0.0030 - val_accuracy: 0.9993 - val_loss: 0.0027
Epoch 8/10
9951/9951 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9992 -

In [24]:
# Exibindo os resultados
print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'AUC-ROC: {roc_auc:.4f}')

Acurácia: 0.9745
Precisão: 0.0096
Recall: 0.1471
F1-Score: 0.0180
AUC-ROC: 0.5265


In [25]:
# Calculando a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Definindo os rótulos
labels = ['Classe 0', 'Classe 1']

# Criando a figura
fig = ff.create_annotated_heatmap(z=cm, x=labels, y=labels, colorscale='Blues', showscale=True)

fig.update_layout(
    title='Matriz de Confusão - Modelo 2',
    xaxis=dict(title='Valores Preditos'),
    yaxis=dict(title='Valores Reais')
)

# Exibindo a matriz
fig.show()

A análise dos resultados indica que o modelo alcançou uma acurácia alta de 0.9745 no conjunto de teste, mas as outras métricas (Precisão, Recall, F1-Score e AUC-ROC) caíram, sugerindo um problema de superajuste (overfitting).

1. **Superajuste (Overfitting):** A acurácia alta combinada com métricas como Precisão (0.0096), Recall (0.1471), e F1-Score (0.0180) extremamente baixas indica que o modelo pode ter se ajustado excessivamente aos dados de treinamento, memorizando padrões específicos e falhando em generalizar para novos dados.

2. **Desequilíbrio de Classes:** A baixa precisão sugere que o modelo pode estar prevendo predominantemente a classe majoritária, resultando em uma alta acurácia enganosa. Em problemas de classificação com classes desbalanceadas, a acurácia não é a métrica mais confiável para avaliar o desempenho.

3. **Avaliação no Conjunto de Teste:** As baixas métricas no conjunto de teste indicam que o modelo não está generalizando bem.


# Conclusão

Apresenta-se abaixo uma análise comparativa dos resultados obtidos com diferentes abordagens para o modelo.

### 1. Modelo Sem Balanceamento das Classes

- **Acurácia**: 0.9994
- **Precisão**: 0.8188
- **Recall**: 0.8309
- **F1-Score**: 0.8248
- **AUC-ROC**: 0.9899

### 2. Modelo Com Balanceamento das Classes

- **Acurácia**: 0.9971
- **Precisão**: 0.3362
- **Recall**: 0.8529
- **F1-Score**: 0.4823
- **AUC-ROC**: 0.9775

### 3. Modelo Com Hiperparâmetros Otimizados

- **Acurácia**: 0.9745
- **Precisão**: 0.0096
- **Recall**: 0.1471
- **F1-Score**: 0.0180
- **AUC-ROC**: 0.5265

### Análise Comparativa

#### **Acurácia**

- **Modelo Sem Balanceamento**: Acurácia extremamente alta (0.9994), indicando que o modelo pode estar fortemente inclinado a prever a classe majoritária, o que é comum em problemas de classificação desbalanceada.
- **Modelo Com Balanceamento**: Acurácia de (0.9971), que ainda é alta, mas a redução sugere que o balanceamento das classes teve um impacto.
- **Modelo Com Hiperparâmetros Otimizados**: Acurácia ainda mais baixa (0.9745), indicando que o processo de otimização dos hiperparâmetros pode não ter sido eficaz para este modelo.

#### **Precisão**

- **Modelo Sem Balanceamento**: Precisão alta (0.8188), sugerindo que o modelo é eficaz em identificar corretamente as transações fraudulentas, mas com risco de prever predominantemente a classe majoritária.
- **Modelo Com Balanceamento**: Queda na precisão (0.3362), refletindo um aumento nos falsos positivos. Isso é comum ao aplicar técnicas de balanceamento, que melhoram o recall, mas muitas vezes reduzem a precisão.
- **Modelo Com Hiperparâmetros Otimizados**: Precisão extremamente baixa (0.0096), o que indica que a maioria das previsões positivas está incorreta.

#### **Recall**

- **Modelo Sem Balanceamento**: Recall alto (0.8309), sugerindo que o modelo está capturando a maioria das transações fraudulentas, mas possivelmente em detrimento da classe minoritária.
- **Modelo Com Balanceamento**: Recall de (0.8529), mostrando que o balanceamento ajudou o modelo a identificar mais transações fraudulentas, embora à custa de uma menor precisão.
- **Modelo Com Hiperparâmetros Otimizados**: Recall muito baixo (0.1471), indicando que o modelo está falhando em detectar a maior parte das fraudes.

#### **F1-Score**

- **Modelo Sem Balanceamento**: F1-Score alto (0.8248), equilibrando bem precisão e recall, adequado para a avaliação do desempenho geral do modelo.
- **Modelo Com Balanceamento**: F1-Score reduzido (0.4823), refletindo o trade-off entre precisão e recall após o balanceamento das classes.
- **Modelo Com Hiperparâmetros Otimizados**: F1-Score muito baixo (0.0180), indicando um desempenho muito ruim do modelo em termos de equilíbrio entre precisão e recall.

#### **AUC-ROC**

- **Modelo Sem Balanceamento**: AUC-ROC muito alta (0.9899), sugerindo uma excelente capacidade do modelo de distinguir entre classes, mas pode ser enganoso em contextos de classes desbalanceadas.
- **Modelo Com Balanceamento**: AUC-ROC ainda alta (0.9775), apesar de uma leve queda, indicando que o balanceamento das classes impactou a discriminação geral do modelo.
- **Modelo Com Hiperparâmetros Otimizados**: AUC-ROC mais baixa (0.5265), mostrando que o processo de otimização de hiperparâmetros resultou em um modelo com capacidade muito reduzida para distinguir entre classes.

### Conclusão

- **Modelo Sem Balanceamento**:Mostra excelente desempenho em termos de acurácia, precisão e recall, mas deve-se tomar cuidado com a possibilidade de que essa performance esteja sendo pela predominância da classe majoritária. No entanto, entre os modelos analisados, ele ainda oferece os melhores resultados para esse conjunto de dados, mantendo um equilíbrio entre as métricas.

- **Modelo Com Balanceamento**: Embora tenha melhorado o recall, essa melhoria veio à custa de uma significativa perda na precisão. Isso resulta em um F1-Score mais baixo, indicando que, embora o modelo esteja capturando mais fraudes, ele também está gerando mais falsos positivos, o que pode ser problemático em cenários de aplicação real.

- **Modelo Com Hiperparâmetros Otimizados**: Os resultados mostram que a otimização dos hiperparâmetros não foi eficaz, levando a um desempenho inferior em todas as métricas principais.

### Qual é o Melhor Modelo para Este Conjunto de Dados?

Com base nas métricas avaliadas, o **Modelo Sem Balanceamento das Classes** é o mais adequado para este conjunto de dados. Embora exista a possibilidade de ele estar influenciado pela predominância da classe majoritária, ele ainda proporciona o melhor equilíbrio entre precisão, recall, F1-Score e AUC-ROC, tornando-o a opção mais robusta para detecção de fraudes neste cenário.